In [6]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
max_steps = 1000
learning_rate = 0.001
dropout = 0.9
data_dir = 'MNIST_data'
log_dir = 'MNIST_data/logs/mnist_with_summaries'

mnist = input_data.read_data_sets(data_dir,one_hot=True)
sess = tf.InteractiveSession()

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y_ = tf.placeholder(tf.float32,[None,10],name='y-input')
    
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shaped_input,10)

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)

In [6]:
def nn_layer(input_tensor,input_dim,output_dim,layer_name,act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim,output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor,weights) + biases
            tf.summary.histogram('pre_activation',preactivate)
        activations = act(preactivate,name='activation')
        tf.summary.histogram('activations',activations)
        return activations

In [7]:
hidden1 = nn_layer(x,784,500,'layer1')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probaility',keep_prob)
    dropped = tf.nn.dropout(hidden1,keep_prob)
    
y = nn_layer(dropped,500,10,'layer2',act=tf.identity)
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=y,labels=y_)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy',cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
tf.summary.scalar('accuracy',accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [12]:
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + '/train',sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test')
tf.global_variables_initializer().run()

In [13]:
def feed_dict(train):
    if train:
        xs,ys = mnist.train.next_batch(100)
        k = dropout
    else:
        xs,ys = mnist.test.images,mnist.test.labels
        k = 1.0
    return {x:xs,y_:ys,keep_prob:k}

In [14]:
saver = tf.train.Saver()
for i in range(max_steps):
    if i % 10 == 0:
        summary,acc = sess.run([merged,accuracy],feed_dict=feed_dict(False))
        test_writer.add_summary(summary,i)
        print('Accuracy at step %s : %s' % (i,acc))
    else:
        if i % 100 == 99:
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary,_ = sess.run([merged,train_step],feed_dict=feed_dict(True),
                                options=run_options,run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata,'step%03d'%i)
            train_writer.add_summary(summary,i)
            saver.save(sess,log_dir+"/model.ckpt",i)
            print('Adding run metadata for', i)
        else:
            summary,_ = sess.run([merged,train_step],feed_dict=feed_dict(True))
            train_writer.add_summary(summary,i)
train_writer.close()
test_writer.close()

Accuracy at step 0 : 0.095
Accuracy at step 10 : 0.7409
Accuracy at step 20 : 0.8434
Accuracy at step 30 : 0.8661
Accuracy at step 40 : 0.8883
Accuracy at step 50 : 0.8956
Accuracy at step 60 : 0.898
Accuracy at step 70 : 0.9087
Accuracy at step 80 : 0.9094
Accuracy at step 90 : 0.9155
Adding run metadata for 99
Accuracy at step 100 : 0.9144
Accuracy at step 110 : 0.9216
Accuracy at step 120 : 0.9223
Accuracy at step 130 : 0.9275
Accuracy at step 140 : 0.9284
Accuracy at step 150 : 0.9311
Accuracy at step 160 : 0.9257
Accuracy at step 170 : 0.9284
Accuracy at step 180 : 0.928
Accuracy at step 190 : 0.9319
Adding run metadata for 199
Accuracy at step 200 : 0.9303
Accuracy at step 210 : 0.9335
Accuracy at step 220 : 0.9387
Accuracy at step 230 : 0.9358
Accuracy at step 240 : 0.9399
Accuracy at step 250 : 0.9387
Accuracy at step 260 : 0.9431
Accuracy at step 270 : 0.9443
Accuracy at step 280 : 0.9404
Accuracy at step 290 : 0.9452
Adding run metadata for 299
Accuracy at step 300 : 0.9466
A